In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# Define the CNN model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.fc = nn.Linear(32 * 8 * 8, 10)  # 32 channels * 8x8 image size

    def forward(self, x):
        out = self.conv1(x)
        out = self.relu(out)
        out = self.maxpool(out)
        out = self.conv2(out)
        out = self.relu(out)
        out = self.maxpool(out)
        out = out.view(out.size(0), -1)  # Flatten the tensor
        out = self.fc(out)
        return out

# Set device to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyperparameters
num_epochs = 5
batch_size = 32
learning_rate = 0.001

# Load the CIFAR10 dataset and perform data augmentation
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Initialize the model
model = CNN().to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{total_step}], Loss: {loss.item():.4f}')

# Testing the model
model.eval()  # Evaluation mode (e.g. disable dropout)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
    print(f'Test Accuracy: {(100 * correct / total):.2f}%')

# Save the model checkpoint
torch.save(model.state_dict(), 'cnn_model.ckpt')

Files already downloaded and verified
Files already downloaded and verified
Epoch [1/10], Step [100/1563], Loss: 2.1984
Epoch [1/10], Step [200/1563], Loss: 2.0069
Epoch [1/10], Step [300/1563], Loss: 1.7245
Epoch [1/10], Step [400/1563], Loss: 1.7906
Epoch [1/10], Step [500/1563], Loss: 1.9716
Epoch [1/10], Step [600/1563], Loss: 1.7227
Epoch [1/10], Step [700/1563], Loss: 1.6656
Epoch [1/10], Step [800/1563], Loss: 1.7107
Epoch [1/10], Step [900/1563], Loss: 1.4670
Epoch [1/10], Step [1000/1563], Loss: 1.7241
Epoch [1/10], Step [1100/1563], Loss: 1.6063
Epoch [1/10], Step [1200/1563], Loss: 1.0467
Epoch [1/10], Step [1300/1563], Loss: 1.2857
Epoch [1/10], Step [1400/1563], Loss: 1.1436
Epoch [1/10], Step [1500/1563], Loss: 1.2538
Epoch [2/10], Step [100/1563], Loss: 1.0050
Epoch [2/10], Step [200/1563], Loss: 1.2317
Epoch [2/10], Step [300/1563], Loss: 1.4810
Epoch [2/10], Step [400/1563], Loss: 1.3186
Epoch [2/10], Step [500/1563], Loss: 1.4248
Epoch [2/10], Step [600/1563], Loss: 1